#### Connect the MySQL database ####

In [41]:
import mysql.connector
import pandas as pd
import numpy as np
import torch

In [11]:
connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='@America155088',
            database='PPT'
            )

In [12]:
query = """ SELECT * FROM posts_details"""

In [13]:
df = pd.read_sql(query, connection)

/var/folders/xp/1gy0dsh531s017q00bdzwmgw0000gn/T/ipykernel_80978/1848774206.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [14]:
df.head()

,id,title,author,date,content,comments
0,1,Re: [問卦] 以前當兵都是一起洗澡的嗎？？？,ai2311 (),Fri Apr 19 21:18:58 2024,"['※ 引述《gn505250 (dwas356916)》之銘言：', ': 聽說以前當兵都...",
1,2,[問卦] 你們股票都有賺到吧？,opemhood (大漢堡),Fri Apr 19 21:21:24 2024,欸欸最近八年來，股市一直破新高這八年，鄉民都有賺到錢吧？一定有賺到錢吧？有沒有這方面的八卦？--,
2,3,[問卦] 歸０是什麼感覺,vigle5566 (威哥),Fri Apr 19 21:20:11 2024,搭給賀 我天橋下肥宅拉阿肥友人是一位玩遊戲會怒吼亂搓的豬角色的操控很弱 技能使出來的叭叭會被...,
3,4,[問卦] 凱格爾訓練機幹嘛用的？,cowardlyman (有功夫無懦夫),Fri Apr 19 21:17:50 2024,本肥被推薦購買這個東西這個東西是要訓練什麼看那顆球的位置是要訓練肛門讓括約肌變有力嗎？有卦嗎？--,
4,5,"[問卦] 台灣空軍飛官,有人遇過UFO嗎?",lelena (太陽神-RA),Fri Apr 19 21:17:26 2024,"遇到UFO的飛行員,都是國外的飛行員,台灣飛行員也是每天在空中飛,不知道有沒有台灣飛官遇過不...",


#### Install NLP model first
#### USing Bert Chinese model

In [67]:
from torch.nn.functional import softmax

In [98]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline
model = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
text_classification = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#### Save it in local, so we don't need to install it everytime

In [99]:
model_path = './bert-base-chinese-model'
tokenizer_path = './bert-base-chinese-tokenizer'

# 保存模型和分詞器
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('./bert-base-chinese-tokenizer/tokenizer_config.json',
 './bert-base-chinese-tokenizer/special_tokens_map.json',
 './bert-base-chinese-tokenizer/vocab.txt',
 './bert-base-chinese-tokenizer/added_tokens.json',
 './bert-base-chinese-tokenizer/tokenizer.json')

In [71]:
import re

In [ ]:
def classify_content(text):
    text = re.sub(r"[^\u4e00-\u9fff\d.a-zA-Z%+\-。！？，、；：（）【】《》“”‘’]", '', text)  # 去除特殊字符
    return text_classification(text)

In [124]:
from transformers import pipeline

# 選擇一個預訓練的摘要模型
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Your max_length is set to 130, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)


 鄉民都有賺到錢吧？   有沒有�’方面的八卦 ？ “Youth’s future” is a term used to refer to young people in the United States.


In [ ]:
# 文章內容
article = """
欸欸最近八年來，股市一直破新高這八年，鄉民都有賺到錢吧？一定有賺到錢吧？有沒有這方面的八卦？
"""

# 生成摘要
summary = summarizer(article, max_length=130, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

In [137]:
def summary_content(text):
    text = re.sub(r"[^\u4e00-\u9fff\d.a-zA-Z%+\-。！？，、；：（）【】《》“”‘’]", '', text) 
    try:
        summary = summarizer(text, max_length=400, min_length=30, do_sample=False)
    except IndexError:
        return text
        
    return summary[0]['summary_text']

In [ ]:
from gensim.summarization import keywords

### TF-IDF

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [155]:
# 創建一個TF-IDF Vectorizer對象
tfidf = TfidfVectorizer()


In [161]:
documents = []

In [162]:
documents.append(x for x in df['content'])

In [168]:
for text in df['content']:
    text = re.sub(r"[^\u4e00-\u9fff\d.a-zA-Z%+\-。！？，、；：（）【】《》“”‘’]", '', text)
    documents.append(text)

#### Different Tokenize

In [122]:
import jieba

In [140]:
for token in jieba.cut("欸欸最近八年來，股市一直破新高這八年，鄉民都有賺到錢吧？一定有賺到錢吧？有沒有這方面的八卦？"):
    print(token)

欸
欸
最近
八年
來
，
股市
一直
破
新高
這
八年
，
鄉民
都
有
賺
到
錢
吧
？
一定
有
賺
到
錢
吧
？
有
沒
有
這
方面
的
八卦
？
